### Installing required libraries

In [ ]:
!pip install numpy-stl matplotlib ipywidgets vtkplotlib

### Importing libraries

In [2]:
import math
from typing import List
from stl.mesh import Mesh
import functools
from stl.mesh import Mesh
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import ipywidgets as widgets
from IPython.display import clear_output
from os import walk
import numpy as np
import vtkplotlib as vpl

### Searching for models in directory

In [3]:
models = []
for (dirpath, dirnames, filenames) in walk("models"):
    models.extend(filenames)
    break

### Getting index of the axis left after selecting the spectator's plane

In [4]:
def get_spectator(mode: str) -> int:
    m = mode.lower()
    if m == "zy" or m == "yz":
        return 0
    elif m == "xz" or m == "zx":
        return 1
    elif m == "xy" or m == "yx":
        return 2

### Sorting normals, selecting only needed, mapping them to the size of the area they are attached to

In [5]:
def sort_normals(normals: List[list], areas: List[list], mode: str) -> List[tuple]:
    result = []
    scalar_index = get_spectator(mode)
    for i in range(len(normals)):
        if normals[i][scalar_index] < 0:
            result.append((normals[i][scalar_index], areas[i][0]))

    return result

### Processing selected normals, so they will form an array of amount of light reflecting from each polygon

In [6]:
def start_processing(income: Mesh, mode: str, rotate_params: list, steps: int, multiplicator: float) -> list:
    result = []
    for x in range(steps):
        income.rotate(rotate_params, math.radians(multiplicator))
        result.append(functools.reduce(lambda z, y: z + y, list(map(lambda t: t[0] * t[0] * t[1], sort_normals(income.get_unit_normals(), income.areas, mode)))))

    return result

### Normalizing values

In [7]:
def normalize(income: list) -> list:
    m = max(income)
    return list(map(lambda x: x / m, income))

### Setting Jupyter widgets up

In [8]:
wx = widgets.FloatSlider(
    value=-1,
    min=-1,
    max=1,
    step=0.1,
    description='x:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f'
)

wy = widgets.FloatSlider(
    value=-1,
    min=-1,
    max=1,
    step=0.1,
    description='y:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f'
)

wz = widgets.FloatSlider(
    value=0.5,
    min=-1,
    max=1,
    step=0.1,
    description='z:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f'
)

w_xy = widgets.Dropdown(
    options=['xy', 'xz', 'yz'],
    value='xy',
    description='Plane:',
    disabled=False,
)

w_files = widgets.Dropdown(
    options=models,
    value=models[0],
    description='Model:',
    disabled=False,
)

button = widgets.Button(description="Show picture")

output = widgets.Output()

def on_button_clicked(b):
    with output:
        clear_output(True)
        x = wx.value
        y = wy.value
        z = wz.value
        model = w_files.value
        xyz = w_xy.value
        process("models/" + model, [x, y, z], xyz)

### Running main processing function

In [9]:
def process(filename: str, rotate: list, mode: str):
    mesh = Mesh.from_file(filename)
    period = 360
    step = 1
    steps = period/step
    figure(figsize=(10, 5), dpi=80)
    nd = normalize(start_processing(mesh, mode, rotate, int(steps), step))
    nd.append(nd[0])
    plt.plot(range(int(steps+1)), nd, color="red")
    plt.yscale('linear')
    plt.xlabel(str(filename[7:len(filename)-4]))
    plt.xticks(range(0, 361, 60))
    plt.show()

### Displaying results after invoking all needed functions

In [10]:
display(wx, wy, wz, w_xy, w_files, button)
button.on_click(on_button_clicked)

with output:
    process("models/" + models[0], [-1, -1, 0.5], "xy")

output

FloatSlider(value=-1.0, continuous_update=False, description='x:', max=1.0, min=-1.0, readout_format='.1f')

FloatSlider(value=-1.0, continuous_update=False, description='y:', max=1.0, min=-1.0, readout_format='.1f')

FloatSlider(value=0.5, continuous_update=False, description='z:', max=1.0, min=-1.0, readout_format='.1f')

Dropdown(description='Plane:', options=('xy', 'xz', 'yz'), value='xy')

Dropdown(description='Model:', options=('golevka.stl', 'hw1.stl', 'kleopatra.stl', 'mithra.stl', 'sphere.stl',…

Button(description='Show picture', style=ButtonStyle())

Output()